In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
from datetime import datetime
import pandas as pd
from collections import defaultdict
from IPython.display import display

In [ ]:
# Load the model and tokenizer
model_name = "meta-llama/Llama-3.2-1B"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Set pad_token and pad_token_id if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Load the model with appropriate precision
precision = torch.float16 if torch.cuda.is_available() else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=precision,
    device_map='auto'
).to(device)

model.safetensors:  75%|#######5  | 1.86G/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# Candidate input sentences
candidate_sentences = [
    "Artificial intelligence has transformed the way we ",
    "The city of Seoul is known for its vibrant culture and ",
    "Hanyang University is a leading institution for "
]

In [ ]:
def generate_text(input_text, input_max_length=256, output_max_length=256, num_return_sequences=1, **kwargs):
    tokenized_inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=input_max_length,
        truncation=True,
        padding=True
    )
    input_ids = tokenized_inputs.input_ids.to(device)
    attention_mask = tokenized_inputs.attention_mask.to(device)
    try:
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_new_tokens=output_max_length,
                num_return_sequences=num_return_sequences,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3,
                **kwargs
            )
        decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        # Ensure that the output ends with a complete sentence
        complete_texts = []
        for text in decoded_outputs:
            # To ensure complete sentences
            last_punct = max(text.rfind('.'), text.rfind('!'), text.rfind('?'))
            if last_punct != -1:
                text = text[:last_punct+1]
            complete_texts.append(text)
        return complete_texts
    except RuntimeError as e:
        return [f"Error during generation: {e}"]

In [ ]:
# Define decoding strategies
decoding_strategies = [
    ("Greedy Decoding", {}),
    ("Beam Search Decoding", {"num_beams": 5, "early_stopping": True}),
    ("Top-k Sampling", {"do_sample": True, "top_k": 50}),
    ("Top-p Sampling", {"do_sample": True, "top_p": 0.9}),
    ("Random Sampling with Temperature", {"do_sample": True, "temperature": 1.5})
]

In [ ]:
# Generate and store results for each candidate sentence and strategy
results = defaultdict(dict)
for sentence in candidate_sentences:
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            sentence,
            input_max_length=256,
            output_max_length=128,
            num_return_sequences=1,
            **strategy_params
        )
        results[sentence][strategy_name] = generated_texts[0] if generated_texts else ""

In [ ]:
# Adjust display settings for better readability
pd.set_option('display.max_colwidth', None)

In [ ]:
# Display results
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.sort_index(inplace=True)  # Sort the DataFrame by index for better readability

In [ ]:
display(results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
Artificial intelligence has transformed the way we,"Artificial intelligence has transformed the way we live, work and play. It is reshaping our economy as well.\nThe latest technology in artificial intelligence (AI) can be used to help detect cancer at its earliest stage, reduce surgical errors by up to a third, improve medical diagnosis and increase accuracy of drug discovery.\nHowever, AI technologies are also raising concerns about privacy, bias and cybersecurity. The EU’s Digital Services Act aims to address these challenges while ensuring that new digital services contribute positively to society.","Artificial intelligence has transformed the way we 3D print objects, but it’s not without its challenges. In this blog post, we’ll explore some of the most common issues that arise when using artificial intelligence for 3d printing, and how to overcome them.\nOne of the biggest challenges with artificial intelligence is the need for large amounts of data. In order to train an AI model, it needs to be fed a large amount of data, which can be difficult to come by. This can lead to overfitting, where the model becomes too specific to the data it’s been trained on and doesn’t generalize well to new data.","Artificial intelligence has transformed the way we 3D print, but what about in manufacturing? Will AI be able to transform industrial production too?\nWith increasing levels of automation and computerization taking place across many industries today there is no doubt that artificial intelligence (AI) will play a key role. But how far can it go before its limitations are exposed by human error or inefficiencies.\nOne area where this technology could have an impact on manufacturers is with additive manufacturing processes such as 3d printing. These techniques involve creating objects layer-by-layer using various materials including metals, plastics and composites.","Artificial intelligence has transformed the way we think and work. From driving self-driving cars to delivering personalized health care, AI is changing our lives in ways that were unimaginable just a few years ago.\nBut what about humans? Are they ready for this new era of artificial intelligence?\nThe answer may surprise you: Yes!","Artificial intelligence has transformed the way we use words. When computers became intelligent enough, in our mind’s eye, they began to write for us.\nSome saw it as a gift from God with promises of salvation and redemption via machines that could think out complicated problem-solving algorithms – an AI paradise compared with today.\nBut artificial intelligence is more like being thrown down into hell where anything goes once its free-floating brain is unleashed — whether you know what it was built on or not.\nWhile we try so longed hard all kinds of reasons why this doesn’t deserve attention outside certain corners; there are too many examples within human beings."
Hanyang University is a leading institution for,"Hanyang University is a leading institution for 21st century higher education in Korea. Founded as the first public university of its kind, Hanyang has grown into one of the most prestigious universities with over 30 colleges and schools spanning across six campuses.\nWith more than 22,000 students enrolled at any given time (including international student population), it’s easy to see why this school consistently ranks among top institutions worldwide! With cutting-edge research facilities like their Center For Excellence In Nanoscience & Technology where they conduct some incredible scientific experiments involving quantum physics; there are plenty opportunities here too!","Hanyang University is a leading institution for 21st century education and research in Korea. It was founded in 1946 as the first private university in Korea, and has since grown to become one of the largest and most prestigious universities in the countr

In [ ]:
# List of complex questions
complex_questions = [
    "Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?",
    "Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?"
]

In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            question,
            input_max_length=512,
            output_max_length=512,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(question):].strip()

        # Optionally, ensure the answer ends with a complete sentence
        last_punct = max(generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1]

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

In [ ]:
# Display results for complex questions
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)  # Sort the DataFrame by index for better readability

In [ ]:
display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?",(Assume that there is no overlap between two successive plantings.,"A: The number of plants after the drought is 30 - 40 = 10, and the number of new plants is 50 - 10 = 40. Therefore, there are 50 plants in total.\nQ: There are 20 students in a class. If 2 students are selected at random, what is the probability that both of them are boys?","(Round your answer to two decimal places.)\nA: The number of total plant is given as\n\(\displaystyle{N = \frac{{1}}{{2}}}\left( {{T - D} + {D - C}{P_0}} \right)\)\nwhere T=total no.of plants after planting,\nC=constant for all cases.\nHere, N = Total No.",(Round your answer to two decimal places.,
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,"A) 100 B) 110 C) 120 D) 130 E) 140\nAnswer (E)\nEach marker is used twice, so the total number of usable markers in each box are:\nBox #1 - Used once -> Unused = 19 markers.\nBox #2 - Used two times -> Unused=12markers\nTotal unused markers per Box = Total Number Of Markers In The Boxes-Used Once/- Twice Per Box --> (20*2+18)=38; thus, there are a total of 4x3=12 empty boxes or an equal amount of markers that can be used by all students at school.\nNow, we know how much he has given away as well as what's remaining. So, using this information, let us solve for x, which represents ""how many markers"" available to him now...\nWe will use the formula below to find out if our answer makes sense!\n\begin{aligned} \frac{x}{40}=0.",A: The number of markers Mike has left is 20 - 15 = 5.,"A) 10 B) 8 C) 6 D) 2 E) 1\nAnswer : The question is asking for the number of remaining markers, so we are looking at how much there were in each box.\nEach box contained 20 Markers.\nMike Gave away 25 Markers (to a classmate).","(1 mark)\nA: We can use the formula below.\n\begin{align*} \frac{x}{b}-c = d \\ x= b-c-d\\ \end{align}\nWe know that there are $x$ marks in total, so we replace it with this value:\n$x=\left(20-30+15-\right)$\nThis gives us a number between zero and twenty-five which is not possible since he already used up all those numbers!\nThe answer must be less than or equal to fifteen!","(4) pts.\nAnswer : Let's call the # of boxes ""x""\n#of Markers in one Box = 0 < x <=24 --> Solve algebraic equation for 'k'. There are two roots so choose either negative( k= -16 ) or positive('k') (-12)\nTherefore, Q is therefore correct and we've got a range [6-3]. Answer options B & C make us change 'Box' into numbers\nE is good also because it matches up with what i had written down above but I do like having A + F just be the answers, they don't matter much anymore by themselves after doing multiple problems, and more info helps.\nHere're some extra examples:\nThe problem was easy as long time ago..."


In [ ]:
# Extract and display only the Beam Search Decoding results
beam_search_results = {question: results['Beam Search Decoding'] for question, results in complex_results.items()}

# Create a DataFrame for Beam Search Decoding results
beam_search_df = pd.DataFrame.from_dict(beam_search_results, orient='index', columns=['Beam Search Decoding'])

display(beam_search_df)

,Beam Search Decoding
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,A: The number of markers Mike has left is 20 - 15 = 5.
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?","A: The number of plants after the drought is 30 - 40 = 10, and the number of new plants is 50 - 10 = 40. Therefore, there are 50 plants in total.\nQ: There are 20 students in a class. If 2 students are selected at random, what is the probability that both of them are boys?"


In [ ]:
# Prepare few-shot examples
few_shot_prompt = """

Q: Lily has 4 boxes of crayons. Each box contains 18 crayons. She gave 15 crayons to her friend and lost 9 crayons. How many crayons does Lily have left?
A: 48 crayons.

Q: John bought 3 packs of pencils. Each pack has 12 pencils. He got 10 pencils from his sister and lost 5 pencils. How many pencils does John have left?
A: 41 pencils.

"""


In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the prompt with the current question
    prompt = few_shot_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=1024,
            output_max_length=128,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

In [ ]:
# Display results for complex questions
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)  # Sort the DataFrame by index for better readability

display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?",,100 plants,,,"25\n\n\n\n## Solution\nI would divide total (before any losses) by what is divided up per item; these can then just be combined easily.\nSo, \n$30 \div 2 =20$\nso that after 60%, all things remaining constant it will only take half again for all items as much space, i.e., $$\frac {6}{17}=35$$. Since no other factors change in our situation above,\nthe answers are about equal here."
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,,50 markers.,35 markers.,,


In [ ]:
# Prepare the one-shot example with chain-of-thought reasoning
one_shot_prompt = """Solve the following problem step-by-step:

Q: Lily had 4 boxes of crayons, and each box contained 18 crayons. She gave 15 crayons to her friend and lost 9 crayons. How many crayons does Lily have left?
A:
Let's go through this step-by-step:
1. Lily has 4 boxes of crayons, each containing 18 crayons, so she starts with 4 x 18 = 72 crayons.
2. She gives 15 crayons to her friend, leaving her with 72 - 15 = 57 crayons.
3. She loses 9 crayons, leaving her with 57 - 9 = 48 crayons.
Therefore, Lily has 48 crayons left.

"""

In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the prompt with the current question
    prompt = one_shot_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=1024,
            output_max_length=256,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

In [ ]:
# Display results for complex questions
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)  # Sort the DataFrame by index for better readability

display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?","To find out how much corn there is after planting more crops, we need to subtract from the number of original corn plants before any dieing occurred (the total number of initial plantings) and then add back on some extra crop that was grown as compensation for those who were killed by death due to lack of water during the previous growing season. This means that our answer should be equal to the sum of both numbers minus their difference; or $${\displaystyle {\frac {30*11}{5}}-{\frac {40}{6}}}$$\n$${\text{initial}}=30\times \left({\begin{matrix}{{10}\over{5}}\\ {{8}\over {5}}\\\end{matrix}}\right)=300.$$\nAfter the loss caused by dying plants, the remaining population becomes $$25(0)+12+20=\boxed{67}$$. We can calculate it using the same formula above because all terms are integers since they involve only whole numbers.\nThis method works well if you know an equation involving subtraction and addition without having to do long division first, such as multiplying two fractions together to get another fraction whose denominator is common between them.","The number of plants before the drought is 30 x 10 = 300.\nAfter the drought, there are 300 - 40 = 260 plants left.\nTo find the number of new plants, we need to know how many plants died and how many new ones were planted. We can use the formula:\nNew plants = Old plants - Dying plants\nIn this case, we know that 40 died and 50 were planted, so the new plants are 50 + 50 = 100. Therefore, the farmer now has 260 - 100 = 160 corn plants.","The number of corn plant before is $30 \times 10$, which equals $300$\nAfter the death of some plants, we should deduct that amount from our total; thus,\n$\left(\right. 0. 5 \cdot 100 + 200 \left.",,Here is another way: we will add these together to make our answer easier :\nWe need first find how much land there are before planting that equals outland more or less.
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,Mike started out with a total of 100 markers in all five boxes. After giving away some he now only has 85 markers remaining (total of 80 for him + 95 from his classmate).\nSo we can write an equation like that\n$50+X=70$\nwhere X is how much Markers Mike still has after everything was given away or divided between them. We know what values are possible because they were stated as integers by us earlier on when it came up in the context of these questions about numbers being divisible by other things etc...\nWe solve our equation using addition/subtraction instead of multiplication/division here since there aren't any decimals involved anymore...,"Mike starts with a total of 5 x 20 = 100 markers, so he has 100 - 25 = 75 markers.\nHe then gives 25 to his classmate, leaving him with 75 - 10 = 65 markers.\nFinally, he loses 15, so his remaining markers are 65 - 5 = 60 markers.\nHence, Mike has only 60 left.","Mike started out with a total of 5x20=100markers in all. After giving away 25 he now only has 75 (which is not divisible by 10) so we need to find how much was lost\nAfter losing 15 Markers there were remaining 55 which can be divided into two parts that are both less than 30 i.e. either 28 or 27 but cannot be more because if it would then the remainder when dividing them will also become odd number as well. So let us take 29 for sure as the least common multiple between these numbers is equal to their LCM.\nNow since after loosing 15 marks,there remains one part whose sum is greater than 50. This means that its count must be even ie.,say N such that N + 45 > 60.\nSo here I am going to solve using simple method where I'm adding up first 7 terms till I g

In [ ]:
# Prepare few-shot examples with chain-of-thought reasoning
few_shot_with_chain_prompt = """Solve the following arithmetic problems step-by-step:

Q: Lily has 4 boxes of crayons. Each box contains 18 crayons. She gave 15 crayons to her friend and lost 9 crayons. How many crayons does Lily have left?
A:
Let's go through this step-by-step:
1. Lily has 4 boxes of crayons, each containing 18 crayons, so she starts with 4 x 18 = 72 crayons.
2. She gives 15 crayons to her friend, leaving her with 72 - 15 = 57 crayons.
3. She lost 9 crayons, leaving her with 57 - 9 = 48 crayons.
Therefore, Lily has 48 crayons left.

Q: John bought 3 packs of pencils. Each pack has 12 pencils. He gave 10 pencils to his sister and lost 5 pencils. How many pencils does John have left?
A:
Let's go through this step-by-step:
1. John bought 3 packs of pencils with 12 pencils in each pack, so he starts with 3 x 12 = 36 pencils.
2. He gave 10 pencils to his sister, leaving him with 36 - 10 = 26 pencils.
3. He lost 5 pencils, leaving him with 26 - 5 = 21 pencils.
Therefore, John has 21 pencils left.

Q: Jack has 3 bags of marbles. Each bag has 15 marbles. He lost 10 marbles. But he bought 25 more marbles. How many marbles does Jack have left?
A:
Let's go through this step-by-step:
1. Jack has 3 bags of marbles with 15 marbles in each bag, so he starts with 3 x 15 = 4 marbles.
2. He lost 10 marbles, leaving him with 45 - 10 = 35 marbles.
3. He bought 25 more marbles, leaving him with 35 + 25 = 60 marbles.
Therefore, Jack has 60 marbles left.

"""

In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the prompt with the current question
    prompt = few_shot_with_chain_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=2056,
            output_max_length=256,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()
        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

In [ ]:
# Display results for complex questions
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)  # Sort the DataFrame by index for better readability

display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?","The number of corn plant is given by\nNumber of Corn Plants (N) =\n= Number of Corn Planted after Drought – Number of Dead Corn Plant\n\nNow we can solve for N,\nWe get,\n\nSo there are 20 corn plants.","The number of plants in the farmer's field before the drought was 30 x 10 or 300 plants. After the drought, there were 300 - 40 = 260 plants left. Therefore, the farmer now has 260 + 50 = 310 plants in his field.","We can use multiplication here! Let’s start by writing down what we know about how there are total 100 corn plants (there were originally 300).\nNext, let us subtract the number that was lost due to dying (i.e., 20) from our original count. This leaves us with only 80 remaining. Finally, multiply these two numbers together since they represent one group of ten plants per row, giving you an answer of\n\begin{align*} \frac{\left(800\right)^{*}}{6^{0} *} = 600. \\ \end{align*\n\nSo then, if we add up all those groups, it is equal to $120$.\n\nNote: The correct solution uses division instead of subtraction for the final part; however, I found both methods equally easy to understand!\n\n## Related Questions\n\n### In base-ten notation, write out the decimal expansion for $\sqrt[8]{102}$, where $n > 7$\n\nDecimals & Fractions ; Arithmetic Operations on Decimals\n\n### If $x < y$ and $y < z,$ which of the following must be true?",,"First, let's count how big is one cor nplant\nThere are actually two options...\nIf 13 corns were harvested there should be an amount equivalent to that multiplied with three because every single corner grows from it!\nSo I started off by just trying out various multipliers like :((20*11)^(th):D What numbers did i come up?\n\n$ \sqrt{n}=?$\nThe root operation $\left(\left[\right.$ denotes the root function $>$)."
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,"Mike started with a total of 30 markers (each marker is counted as one unit), but after giving away some markers to friends and losing others, he ends up having only\n$$(\text{boxes} \times \text{markers per box}) – (\text{box contents}-\text{s) }$$\n\nThe number of boxes $=$ $$x_0+6(8-7)=11-x_0$$\n\nSince there are five boxes,\n\n$x_0=11$\n\nHence, we get that all remaining numbers should be multiples of 11 or 22 i.e., integers divisible by both 11 & 22.\n\nWe need to find two such values for which the difference between their squares will give us our target value. The sum of digits of these two terms must also equal the original given value, since it can't exceed either 99 or 999.","Here's how we can solve this problem:\nLet’s go through the steps one at a time:\nStep 1: Mike started with 5 x 20 = 100 markers.\nStep 2: He gave away 25 of them to his classmate, leaving 100 - 25= 75 markers. This means that he has 75 – 15= 60 markers left.\nStep3: He lost some markers, so we need to subtract 15 from the number of markers he has left. This leaves us with 60 – 14 = 46 markers.","Mike started by having a total number of 50 markers which is equal to (5x20) + (5×20)=100\nHe then give away 30 of those marksers to his classmate, thus resulting into only remaining 70 Marks after giving them out.\nTo get back that amount,he had to lose 40 Markers.","Here is a list of all possible steps for solving this problem:\nStep #1 : Mike started with 5x20=100 markers\nStep #2 : He gave away 25markers (he can give out as much or as little as he wants), making it now 75-25=50markers.\nStep#3 : H e lo sse d 7marker s. H i c h u p t o m ay be 0-7=-8 marker s remaining.\nHence, we are done by going back one step at a time until you get your answer.\n\n#### Related Posts\n\n### What

In [ ]:
# Load the instruct model and tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# Set pad_token and pad_token_id if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def generate_text(input_text, input_max_length=1024, output_max_length=512, num_return_sequences=1, **kwargs):
    tokenized_inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=input_max_length,
        truncation=True,
        padding=True
    )
    input_ids = tokenized_inputs.input_ids.to(device)
    attention_mask = tokenized_inputs.attention_mask.to(device)
    try:
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=output_max_length,
                num_return_sequences=num_return_sequences,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                **kwargs
            )
        decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        return decoded_outputs
    except RuntimeError as e:
        return [f"Error during generation: {e}"]


In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the few-shot prompt with the current question
    prompt = few_shot_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=512,
            output_max_length=128,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

# Display results
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)
display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?",40,100,50 corn plants.,150 corn plants,35 corn plantsfrom our original calculation 10
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,45 markers.,35 markers.,20 markers.,60 markers.,"50\n\nA: Mark is given three boxes. Each box contains 20 candies.\n\n*Note: Mark cannot have exactly 50 candies.\n\nA: Let X represent Mark's total candies.\n\n*To solve this question, the 50 candies must all be used up:\n\n*First, Mark takes 25 candies. There are 5 boxes to take 25 candies, so each box represents 5 candies. X=20-25=5 25 candies can fit into boxes. Hence a 8 candies leftover"


In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the one-shot prompt(chain-of-thought) with the current question
    prompt = one_shot_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=1024,
            output_max_length=512,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

# Display results
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)
display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. 40 plants died, leaving 300 - 40 = 260 plants.\n3. The farmer planted 50 new ones, so he has 300 + 50 = 350 plants.\n4. Therefore, he has 350 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. After a drought, 40 plants died, so he now has 300 - 40 = 260 plants.\n3. He planted 50 new ones, so he now has 260 + 50 = 310 plants.\nTherefore, the farmer has 310 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. After a drought, 40 plants died, so he now has 300 - 40 = 260 plants.\n3. He planted 50 new ones, so he now has 260 + 50 = 310 plants.\nTherefore, he has 310 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. 40 plants died, leaving 300 - 40 = 260 plants.\n3. The farmer planted 50 new ones, so he now has 260 + 50 = 310 plants.\nTherefore, the farmer has 310 plants.","After the 30 corn plants die, he will have (30 – 40 – 50)/10 + 10 = 6+10 = 16 new ones remaining to plant.\nAfter planting, 16*9 = 144 corn plants can be harvested and counted from each row after the drought.\nHe has, 10 + 144 = 150 total new plants that he planted in new corn rows."
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,"Let's go through this step-by-step:\n1. Mike has 5 boxes of markers, each containing 20 markers, so he starts with 5 x 20 = 100 markers.\n2. He gives 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He loses 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers, each containing 20 markers, so he starts with 5 x 20 = 100 markers.\n2. He gives 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He loses 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers, each containing 20 markers, so he starts with 5 x 20 = 100 markers.\n2. He gives 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He loses 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers, each containing 20 markers, so he starts with 5 x 20 = 100 markers.\n2. He gives 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He loses 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's use this problem and go through it:\n\n1. Mike has 5 x 20 = 100 markers, and each student gives him back 25 markers.\n\n2. He ends up with a total of 100 markers minus 25 = 75 markers. Then he subtracts 15:\n\n(75 - 15) = 60.\n\nIn total there were 100 - 60 + 25 = 25 markers left to be distributed between his classmate.\n\nMike left with 25 / his classmate = $\frac{25}{25}=1 \left(\mathrm{classmate}\right)$ markers each.\n\nSo in total he is left with 25 markers per his classmate:\n\n5 x $\frac{1}{25}$ = 1 (he gave himself $\frac{1}{25}$ = \mathrm{2 markers for himself in the beginning}).\n\nLet's go back:\n\nLet's say, $\mathcal{x}$ = the number of his classmates = the amount of markers which 

In [ ]:
# Generate and store results for each complex question and strategy
complex_results = defaultdict(dict)
for question in complex_questions:
    # Combine the few-shot prompt(chain-of-thought) with the current question
    prompt = few_shot_with_chain_prompt + question + "\nA:\n"
    for strategy_name, strategy_params in decoding_strategies:
        generated_texts = generate_text(
            prompt,
            input_max_length=2056,
            output_max_length=512,
            num_return_sequences=1,
            **strategy_params
        )
        full_output = generated_texts[0] if generated_texts else ""
        # Remove the prompt from the output to get only the answer
        generated_answer = full_output[len(prompt):].strip()

        # Truncate the output if it includes the next question
        truncation_patterns = ['\nQ:', '\nQuestion:', '\nQuestion', 'Q:', 'Question:']
        truncation_indices = [generated_answer.find(pattern) for pattern in truncation_patterns if pattern in generated_answer]
        if truncation_indices:
            first_truncation = min([index for index in truncation_indices if index != -1])
            generated_answer = generated_answer[:first_truncation].strip()

        # Ensure the answer ends with a complete sentence or step
        last_punct = max(generated_answer.rfind('\n'), generated_answer.rfind('.'), generated_answer.rfind('!'), generated_answer.rfind('?'))
        if last_punct != -1:
            generated_answer = generated_answer[:last_punct+1].strip()

        # Store the result using the question as the key
        complex_results[question][strategy_name] = generated_answer

# Display results
complex_results_df = pd.DataFrame.from_dict(complex_results, orient='index')
complex_results_df.sort_index(inplace=True)
display(complex_results_df)

,Greedy Decoding,Beam Search Decoding,Top-k Sampling,Top-p Sampling,Random Sampling with Temperature
"Q: A farmer has 30 rows of corn plants with 10 plants in each row. After a drought, 40 plants died, but he planted 50 new ones. How many corn plants does the farmer have now?","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. 40 plants died, so he has 300 - 40 = 260 plants.\n3. He planted 50 new plants, so he has 260 + 50 = 310 plants.\nTherefore, the farmer has 310 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. After a drought, 40 plants died, leaving him with 300 - 40 = 260 plants.\n3. He planted 50 new ones, leaving him with 260 + 50 = 310 plants.\nTherefore, the farmer has 310 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. He lost 40 plants, leaving him with 300 - 40 = 260 plants.\n3. He planted 50 new ones, leaving him with 260 + 50 = 310 plants.\nTherefore, the farmer has 310 plants left.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he starts with 30 x 10 = 300 plants.\n2. After a drought, 40 plants died, leaving him with 300 - 40 = 260 plants.\n3. He planted 50 new plants, leaving him with 260 + 50 = 310 plants.\nTherefore, the farmer has 310 corn plants.","Let's go through this step-by-step:\n1. The farmer has 30 rows of corn plants with 10 plants in each row, so he has 30 x 10 = 300 plants.\n2. 40 of his plants died, leaving him with 50 x 10 = 350 fewer plants than he started with, so he only has 300 - 50 = 250 plants now.\n3. Then he planted 50 new corn plants, so his grandtotal is 300 + 50 = 350+50 = 400.\n3. 5\n\n4-5.5%\nThe farmer still has 25 25 rows.\n\n6-7%"
Q: Mike has 5 boxes of markers. Each box contains 20 markers. He gave 25 markers to his classmates and lost 15 markers. How many markers does Mike have left?,"Let's go through this step-by-step:\n1. Mike has 5 boxes of markers with 20 markers in each box, so he starts with 5 x 20 = 100 markers.\n2. He gave 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He lost 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers with 20 markers in each box, so he starts with 5 x 20 = 100 markers.\n2. He gave 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He lost 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers with 20 markers in each box, so he starts with 5 x 20 = 100 markers.\n2. He gave 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He lost 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers with 20 markers in each box, so he starts with 5 x 20 = 100 markers.\n2. He gave 25 markers to his classmates, leaving him with 100 - 25 = 75 markers.\n3. He lost 15 markers, leaving him with 75 - 15 = 60 markers.\nTherefore, Mike has 60 markers left.","Let's go through this step-by-step:\n1. Mike has 5 boxes of markers with 20 markers in each box, so he starts with 5 x 20 = 100 markers.\n2. He gave 25 markers to his classmates, leaving him with 100 - 25 = 100 - 15 = 85 markers.\n3. He lost 15 markers, leaving him with 85 - 15 = 70 markers.\nTherefore, Mike has 85 - 15 = 70 - 15 = 55 markers.\n\n1. How many pens do Bob have left?\n2. What did John give his sister?\n3. How many boxes of marbles do Peter have left?\n4. What are the missing numbers in problem 3?\n5. What are all solutions in cas